In [17]:
# Import necessary libraries for data manipulation and machine learning
import numpy as np  # Used for numerical operations, including array manipulations
import pandas as pd  # Used for data handling and manipulation in DataFrames
from scipy.sparse import csr_matrix  # To handle sparse matrices efficiently for memory usage
from sklearn.neighbors import NearestNeighbors  # For building the nearest neighbors model to recommend books
import matplotlib.pyplot as plt  # For visualizations (not used in this specific code but can be useful for visual analysis)

# Download the dataset containing book information and user ratings
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip  # Download the zip file containing book data
!unzip book-crossings.zip  # Unzip the downloaded file to access the CSV data files

# Define the paths for the CSV files we need to read
books_filename = 'BX-Books.csv'  # File containing book metadata (ISBN, title, author)
ratings_filename = 'BX-Book-Ratings.csv'  # File containing user ratings for books (user, ISBN, rating)

# Load book data into a DataFrame from the CSV file
df_books = pd.read_csv(
    books_filename,  # Path to the books CSV file
    encoding="ISO-8859-1",  # Encoding used in the CSV file to handle non-ASCII characters
    sep=";",  # Separator used in the CSV file (semicolon-separated)
    header=0,  # First row is the header row
    names=['isbn', 'title', 'author'],  # Define column names
    usecols=['isbn', 'title', 'author'],  # Only load the relevant columns
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'}  # Set data types to prevent type conversion issues
)

# Load ratings data into a DataFrame from the CSV file
df_ratings = pd.read_csv(
    ratings_filename,  # Path to the ratings CSV file
    encoding="ISO-8859-1",  # Encoding used to ensure proper handling of special characters
    sep=";",  # CSV separator
    header=0,  # The first row contains column headers
    names=['user', 'isbn', 'rating'],  # Define column names
    usecols=['user', 'isbn', 'rating'],  # Load only the necessary columns
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}  # Set appropriate data types
)

# Filter out users with fewer than 200 ratings and books with fewer than 100 ratings
filter_1 = df_ratings['user'].value_counts()  # Count the number of ratings per user
filter_2 = df_ratings['isbn'].value_counts()  # Count the number of ratings per book
# Apply filters to remove users and books with insufficient data
df_ratings = df_ratings[~df_ratings['user'].isin(filter_1[filter_1 < 200].index) &
                        ~df_ratings['isbn'].isin(filter_2[filter_2 < 100].index)]

# Create a pivot table where rows are books (ISBN), columns are users, and values are ratings
df_table = df_ratings.pivot_table(index='isbn', columns='user', values='rating').fillna(0)

# Map book ISBNs to titles from the 'df_books' DataFrame
df_table.index = df_table.join(df_books.set_index('isbn'))['title']

# Function to generate book recommendations based on cosine similarity
def get_recommends(book=""):
    recommended_books = []  # Initialize an empty list to store recommended books
    # Create a NearestNeighbors model to find books similar to the input book
    nbrs = NearestNeighbors(n_neighbors=6, metric="cosine").fit(df_table.values)
    # Find the 6 nearest neighbors (books similar to the input book)
    distances, indices = nbrs.kneighbors([df_table.loc[book].values], n_neighbors=6)

    # Extract the book titles and distances for the top 5 recommended books (excluding the input book itself)
    for i in range(1, 6):
        recommended_books.append([df_table.index[indices[0][-i]], distances[0][-i]])

    return [book, recommended_books]  # Return the input book and the recommended books

# Test the recommendation function by getting recommendations for a specific book
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)  # Print the recommendations

# Function to test the book recommendation system
def test_book_recommendation():
    test_pass = True  # Flag to track if the test passes
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")  # Get recommendations for a test book

    # Check if the recommended book matches the input book
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False

    # Define the expected recommended books and their distances
    recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]  # Expected cosine distances

    # Verify if the recommended books match the expected ones and if distances are within an acceptable range
    for i in range(4):  # Check the top 4 recommendations
        if recommends[1][i][0] not in recommended_books:
            test_pass = False  # If the recommendation is not expected, fail the test
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:  # Allow a tolerance of 0.05
            test_pass = False  # If distance is too far from expected, fail the test

    # Print the result of the test
    if test_pass:
        print("You passed the challenge!")
    else:
        print("You haven't passed yet. Keep trying!")

# Run the test function
test_book_recommendation()


--2025-03-29 19:50:23--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.15’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2025-03-29 19:50:23 (255 MB/s) - ‘book-crossings.zip.15’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You passed the challenge!
